# Insert data to SQL Server

Files must insert database in specified order, when there are foreign keys between tables. Some parent columns in some tables contains values which must be inserted in before some children column values can insert. Order is next:
1. Alueet (one merged file) and Tieomin (files by years), (FK parent columns 'alue_id', 'tieomin_id' and 'Vuosi' used in Onnettomuudet table).
2. Onnettomuudet (files by year), (FK parent columns 'onnett_id' used in Osalliset table)
3. Osalliset (one merged file)
4. Väestö (one file)

Merged files are created by code file:
- File merge for DB.

In some error situations, the source file or its values may be incorrect. In these situations, the source file may need to be repaired with the following code files:
- Editing Files fo DB , Create File for DB
- Check file columns
- Check file error

In [1]:
# Needed libraries
import pandas as pd
import numpy as np
import pyodbc
import sqlalchemy
from sqlalchemy.engine import URL
from sqlalchemy import create_engine

#### Parameters of DB and Source data

In [2]:
# Database parameters
Database = 'Onnettomuudet'
Tablename = 'Onnettomuudet'

# Source file parameters
src_file_start = '../datasets/onnettomuus/db/tielonnett_'
src_file_end = '_onnett.csv'
years_start = 2020
years_end = 2021
src_delimiter = ';'
src_encoding = 'utf-8'

#### Connection to database

In [2]:
#conn = 'DRIVER={ODBC Driver 17 for SQL Server};Server=localhost\SQLEXPRESS;Database=Onnettomuudet;Trusted_Connection=Yes;'
conn = 'DRIVER={ODBC Driver 17 for SQL Server};server=FUJITSU-PC\SQLEXPRESS;database=Onnettomuudet;trusted_connection=Yes;'
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": conn})
# Create engine between python and database
engine = create_engine(connection_url)

#### Insert one datafile to database

In [ ]:
'''
# Database parameter
one_file_table = 'Osalliset'
# Source file parameters
one_file_start = '../datasets/onnettomuus/db/tielonnett_'
one_file_end = '_osall.csv'
year = 2005

# Read source file 
df = pd.read_csv(one_file_start+str(year)+one_file_end, delimiter=src_delimiter, encoding=src_encoding)

# Insert file to database
# set index=False to avoid bringing the dataframe index in as a column
df.to_sql(one_file_table, con=engine, if_exists='append', index=False)
'''

In [ ]:
#types = dict(df.dtypes)
#print(types)

#max_len = df[df['Oslakpvm'].str.len() == df['Oslakpvm'].str.len().max()]
#max_len['Oslakpvm']
#max_len = max(df['Mo_molsel'], key=len)
#max_len

#### Insert data files to database

In [23]:
#'''
for i in range(years_start,years_end+1):
    df = pd.read_csv(src_file_start+str(i)+src_file_end, delimiter=src_delimiter, encoding=src_encoding)
    df.to_sql(Tablename, con=engine, if_exists='append', index=False)
#'''

#### Replace data to database

In [7]:
# One option is to simply replace the entire old table with an updated table generated from your dataframe:
#df = pd.read_csv('../datasets/onnettomuus/db/tielonnett_alue_all.csv', delimiter=';', encoding='utf-8')
#df.to_sql('Alueet', con=engine, if_exists='replace', index=False)


-1

#### Add data to database

In [3]:
# Second option is to simply append new data to an existing table:
df = pd.read_csv('../datasets/onnettomuus/db/väestö_maakunnittain_db.csv', delimiter=';', encoding='utf-8')
df.to_sql('Väestö', con=engine, if_exists='append', index=False)


-1

#### Check inserting data from table

In [24]:
sqlcommand = 'SELECT COUNT(Onnett_id) AS Onn_count, Vuosi FROM Onnettomuudet GROUP BY Vuosi;'

#sqlcommand = 'SELECT COUNT(alue_id) AS Alue_count FROM Alueet;'

#sqlcommand = 'SELECT COUNT(tieomin_id) AS Tieomin_count FROM Tieomin;'

query = pd.read_sql_query(sqlcommand,con=engine)
df_check = pd.DataFrame(query, columns = ['Vuosi','Onn_count'])
df_check
#df_check_alue = pd.DataFrame(query, columns = ['Alue_count'])
#df_check_alue
#df_check_tieomin = pd.DataFrame(query, columns = ['Tieomin_count'])
#df_check_tieomin

,Vuosi,Onn_count
0,2010,33814
1,2007,16903
2,2021,9567
3,2013,29719
4,2008,17266
5,2016,19991
6,2005,16598
7,2019,15297
8,2014,28345
9,2020,11347


In [27]:
sqlcommand = 'SELECT COUNT(Osall_id) AS Osall_count FROM Osalliset;'

query = pd.read_sql_query(sqlcommand,con=engine)
df_check_osall = pd.DataFrame(query, columns = ['Osall_count'])
df_check_osall

,Osall_count
0,642447


In [ ]:
# Closing database connection
#engine.close()